In [1]:
import os
import sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn

print(torch.__version__)

notebook_dir = os.getcwd()
model_b_root = os.path.abspath(os.path.join(notebook_dir, ".."))
project_root = os.path.abspath(os.path.join(notebook_dir, "../../.."))
print(model_b_root)
print(project_root)

if model_b_root not in sys.path:
    sys.path.append(model_b_root)
    
if project_root not in sys.path:
    sys.path.append(project_root)
    
MODEL_V = "05_simple_transformer"

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device
    

2.2.2
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction/models/model_B
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction


device(type='cpu')

In [2]:
from preprocess import preprocess_data

X_train, X_test, y_train, y_test, patient_ids_train, patient_ids_test = preprocess_data(
                                                                            use_last_processed_data=True,
                                                                            data_file_name="big_imputed_sofa",
                                                                            sampling_method="undersample",
                                                                            train_sample_fraction=0.05)

Project root: /Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction
Loaded from last processed data


In [3]:
X_train.head(5)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SOFA
443,86.0,95.0,37.195388,103.0,68.0,52.000000,16.0,33.055556,-4.106557,23.794118,...,9.668224,45.976531,13.053684,242.0,221.682796,49.02,0,-9.21,12,7
444,81.0,94.0,37.195388,98.0,62.5,48.000000,8.0,33.055556,-4.106557,23.794118,...,9.668224,45.976531,13.053684,242.0,221.682796,49.02,0,-9.21,15,7
445,76.0,100.0,36.440000,118.0,79.0,58.000000,27.0,33.055556,-4.106557,23.794118,...,9.668224,45.976531,13.053684,242.0,221.682796,49.02,0,-9.21,2,6
446,89.0,96.0,37.195388,72.0,74.0,55.344915,14.0,33.055556,-4.106557,23.794118,...,9.668224,45.976531,13.053684,242.0,221.682796,49.02,0,-9.21,23,6
447,89.0,96.0,37.195388,72.0,71.0,55.344915,13.0,33.055556,-4.106557,23.794118,...,9.668224,45.976531,13.053684,242.0,221.682796,49.02,0,-9.21,25,6


In [4]:
patient_ids_train.head(5)

443    14695
444    14695
445    14695
446    14695
447    14695
Name: patient_id, dtype: int64

In [3]:
from custom_dataset import SepsisPatientDataset, collate_fn
from torch.utils.data import DataLoader

dataset = SepsisPatientDataset(X_train.values, y_train.values, patient_ids_train.values)
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [4]:
from architectures import TransformerClassifier
from torch import nn
from training import training_loop

in_dim = X_train.shape[1] 
valid_heads = [h for h in range(1, in_dim + 1) if in_dim % h == 0]
num_heads = valid_heads[-1] 

model = TransformerClassifier(input_dim=in_dim, num_heads=num_heads).to(device)

pos_weight = torch.tensor([2.3], dtype=torch.float32).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

epoch_counter, loss_counter, acc_counter, best_threshold = training_loop(
    model=model,
    train_loader=train_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=10,
    device=device,
    threshold_update_n_batches=10,
)

print(best_threshold)

/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction/venv-sepsis/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/10:   0%|          | 0/16 [00:10<?, ?it/s, Loss=0.98, Acc=0.375, Prec=0.375, Rec=1] 

Epoch 1/10 | Loss: 0.06528 | Accuracy: 0.33% | Precision: 0.28% | Recall: 0.94%


Epoch 2/10 | Loss: 0.06999 | Accuracy: 0.31% | Precision: 0.31% | Recall: 1.00%


Epoch 3/10:   0%|          | 0/16 [00:10<?, ?it/s, Loss=0.76, Acc=0.625, Prec=0.333, Rec=0.5]   

Epoch 3/10 | Loss: 0.05063 | Accuracy: 0.46% | Precision: 0.31% | Recall: 0.66%


...

Epoch 7/10 | Loss: 0.04522 | Accuracy: 0.42% | Precision: 0.34% | Recall: 0.92%


Epoch 8/10 | Loss: 0.04479 | Accuracy: 0.74% | Precision: 0.79% | Recall: 0.38%


Epoch 9/10:   0%|          | 0/16 [00:10<?, ?it/s, Loss=0.93, Acc=0.375, Prec=0.375, Rec=1]     

Epoch 9/10 | Loss: 0.06085 | Accuracy: 0.71% | Precision: 0.66% | Recall: 0.53%


Epoch 10/10 | Loss: 0.04942 | Accuracy: 0.74% | Precision: 0.67% | Recall: 0.49%
0.5


In [5]:
best_threshold

0.5

# Testing loop

In [8]:

import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torchmetrics import Accuracy
from testing import testing_loop


dataset = SepsisPatientDataset(X_test.values, y_test.values, patient_ids_test.values)
batch_size = 32
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

all_y_logits, all_y_probs, all_y_pred, all_y_test = testing_loop(
  model=model,
  test_loader=test_loader,
  loss_fn=loss_fn,
  device=device,
  threshold=best_threshold
)

Test Loss: 0.68204 | Test Accuracy: 0.82%


In [9]:
all_y_logits = torch.cat(all_y_logits).numpy().flatten()
all_y_probs = torch.cat(all_y_probs).numpy().flatten()
all_y_pred = torch.cat(all_y_pred).numpy().flatten()
all_y_test = torch.cat(all_y_test).numpy().astype(int) 

df = pd.DataFrame({
    'y_logits': all_y_logits,
    'y_probs': all_y_probs,
    'y_pred': all_y_pred,
    'y_test': all_y_test
})

# save_eval_csv(df, EVAL)

In [10]:
accuracy = (df['y_pred'] == df['y_test']).mean() * 100
print(f"Total Accuracy: {accuracy:.2f}%")

Total Accuracy: 81.54%


In [ ]:
# from t_utils import save_plots, FEATURE_NAMES, save_metrics

# y_test = df['y_test'].values
# y_probs = df['y_probs'].values
# y_pred = df['y_pred'].values


# save_plots(
#     y_test=y_test,
#     y_probs=y_probs,
#     y_pred=y_pred,
#     model=model, 
#     feature_names=FEATURE_NAMES,
#     eval=EVAL,
#     attention_weights=[]
# )

# save_metrics(
#     y_test=y_test,
#     y_probs=y_probs,
#     y_pred=y_pred,
#     eval=EVAL,
# )